In [ ]:
%%html
<link rel="stylesheet" href="main.css">
<link rel="import" href="urth_components/paper-dropdown-menu/paper-dropdown-menu.html" is='urth-core-import' package='PolymerElements/paper-dropdown-menu'>
<link rel="import" href="urth_components/paper-menu/paper-menu.html" is='urth-core-import' package='PolymerElements/paper-menu'>
<link rel="import" href="urth_components/paper-item/paper-item.html" is='urth-core-import' package='PolymerElements/paper-item'>
<link rel="import" href="urth_components/paper-button/paper-button.html" is='urth-core-import' package='PolymerElements/paper-button'>
<link rel="import" href="urth_components/paper-card/paper-card.html" is='urth-core-import' package='PolymerElements/paper-card'>
<link rel="import" href="urth_components/paper-slider/paper-slider.html" is='urth-core-import' package='PolymerElements/paper-slider'>
<link rel="import" href="urth_components/google-map/google-map.html" is='urth-core-import' package='GoogleWebComponents/google-map'>
<link rel="import" href="urth_components/google-map/google-map-marker.html" is='urth-core-import' package='GoogleWebComponents/google-map'>
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html">
<link rel="import" href="urth_components/urth-viz-chart/urth-viz-chart.html">

<style is="custom-style">
    paper-slider.blue {
      --paper-slider-knob-color: var(--paper-light-blue-500);
      --paper-slider-active-color: var(--paper-light-blue-500);
    }
    paper-slider.red {
      --paper-slider-knob-color: var(--paper-red-500);
      --paper-slider-active-color: var(--paper-red-500);
    }
    paper-slider.orange {
      --paper-slider-knob-color: var(--paper-orange-500);
      --paper-slider-active-color: var(--paper-orange-500);
      --paper-slider-pin-color: var(--paper-orange-500);
    }
    
    urth-viz-chart::shadow .nv-x .tick text {
        display: none;
    }
</style>

In [ ]:
import pandas as pd

In [ ]:
def loadData(name:str):
    return pd.read_csv('./' + name + '.csv')

In [ ]:
%%html
<template id="loadCard" is="urth-core-bind" channel="trips">
    <urth-core-function id="loadDataFunc" 
        ref="loadData" 
        arg-name="{{datafile}}" 
        result="{{thedata}}" 
        limit="50"
        is-ready="{{isloadready}}"></urth-core-function>
    <paper-card heading="Load Data" elevation="1">
        <div class="card-content">
            <paper-dropdown-menu label="Select Trips" selected-item-label="{{datafile}}" noink>
                <paper-menu class="dropdown-content" selected="[[datafile]]" attr-for-selected="label">
                    <template is="dom-repeat" items="[[tripOptions]]">
                        <paper-item label="[[item]]">[[item]]</paper-item>
                    </template>
                </paper-menu>
            </paper-dropdown-menu>
        </div>
        <div class="card-actions">
            <paper-button tabindex="0" disabled="{{!isloadready}}" onClick="loadDataFunc.invoke()">Load</paper-button>
        </div>
    </paper-card>
    <script>
        loadCard.tripOptions = ["trips_1", "trips_2", "trips_3", "trips_4", "trips_5"]
    </script>
</template>

In [ ]:
def center(lat1:float, long1: float, lat2: float, long2: float):
    return [(lat1+lat2)/2, (long1+long2)/2]

In [ ]:
%%html
<template is="urth-core-bind" channel="trips">
    <urth-core-function
        ref="center" 
        arg-lat1="[[sel.11]]" 
        arg-long1="[[sel.10]]" 
        arg-lat2="[[sel.13]]" 
        arg-long2="[[sel.12]]" 
        result="{{thecenter}}" auto></urth-core-function>
    <template is="dom-if" if="{{thedata}}">
        <paper-card style="width: 100%;" heading="Trips Data" elevation="1">
            <div class="card-content">
                <urth-viz-table datarows='{{thedata.data}}' columns='{{thedata.columns}}' selection='{{sel}}' rows-visible="10"></urth-viz-table>
            </div>
            <div class="card-content">
                <template is="dom-if" if="{{sel}}">
                    <google-map id="blah" latitude="[[thecenter.0]]" longitude="[[thecenter.1]]" zoom="12">
                        <google-map-marker latitude="{{sel.11}}" longitude="{{sel.10}}" title="Pickup"></google-map-marker>
                        <google-map-marker latitude="{{sel.13}}" longitude="{{sel.12}}" title="Dropoff"></google-map-marker>
                    </google-map>
                </template>
        </paper-card>
    </template>
</template>

In [ ]:
allfares = fares = pd.read_csv('./fares.csv')

In [ ]:
%%html
<template is="urth-core-bind" channel="faresChan">
    <urth-core-dataframe ref="fares" value="{{thedata}}" limit="100" auto></urth-core-dataframe>
    <paper-card style="width: 100%;" heading="Fares Data" elevation="1">
        <div class="card-content">
            <urth-viz-table datarows='{{thedata.data}}' columns='{{thedata.columns}}' rows-visible='10'></urth-viz-table>
        </div>
    </paper-card>
</template>

In [ ]:
fares = allfares[fares[' payment_type']=='CRD']

In [ ]:
def filterTopEarners(filter = "All"):
    if (filter == "Credit"):
        _fares = allfares[allfares[' payment_type']=='CRD']
    elif (filter == "Cash"):
        _fares = allfares[allfares[' payment_type']=='CSH']
    else:
        _fares = allfares

    _topEarners = pd.DataFrame(_fares.groupby('medallion').sum().sort(' total_amount', inplace=False, ascending=False)[' total_amount']) 
    _topEarners['medallion'] = _topEarners.index
    
    return _topEarners[['medallion', ' total_amount']]

In [ ]:
%%html
<template id="topEarners" is="urth-core-bind" channel="faresChan">
    <urth-core-function id='why' ref="filterTopEarners" arg-filter="{{payType}}" result="{{top}}" limit="{{limit}}" auto></urth-core-function>
    <paper-card style="width: 100%;" heading="Top Earners" elevation="1">
        <div class="card-content">
            <paper-dropdown-menu label="Select Payment Type" noink>
                <paper-menu class="dropdown-content" selected="{{payType}}" attr-for-selected="label">
                    <template is="dom-repeat" items="[[paymentOptions]]">
                        <paper-item label="[[item]]">[[item]]</paper-item>
                    </template>
                </paper-menu>
            </paper-dropdown-menu>

            <urth-viz-chart type='bar' datarows='{{top.data}}' columns='{{top.columns}}' selection="{{sel}}"></urth-viz-chart>
            <div class="center horizontal layout">
                <div>Limit</div>
                <paper-slider min="5" max="50" step="5" pin snap value="{{limit}}"></paper-slider>
            </div>
        </paper-card>
    </div>
    <script>
        topEarners.paymentOptions = ["All", "Cash", "Credit"]
    </script>
</template>

In [ ]:
def driverStats(medallions = []):
    colnames = ['medallion', 'trips', 'total_amount_avg', 'total_amount_min', 'total_amount_max']
    data = []

    for m in list(map((lambda med: med['x']), medallions)):
        stats = allfares[allfares['medallion']==m].groupby('medallion').describe()[' total_amount'].iloc[[0, 1, 3, 7]].values
        row = [m] + list(stats)
        data.append(row)

    return pd.DataFrame(data, columns=colnames)

def minMaxStats():
    data = []
    group = allfares[['medallion', ' total_amount']].groupby('medallion')

    the_mean = group.mean()
    the_max = group.max()
    the_min = group.min()

    return [the_mean.min()[0], the_mean.max()[0], the_max.min()[0], the_max.max()[0], the_min.min()[0], the_min.max()[0]]

In [ ]:
%%html
<template id="fareStatsTemplate" is="urth-core-bind" channel="faresChan">
    <urth-core-function ref="driverStats" arg-medallions="{{sel}}" result="{{stats}}" auto></urth-core-function>
    <urth-core-function id="minMaxStats" ref="minMaxStats" result="{{minMax}}" on-is-ready-changed ="init"></urth-core-function>
    <script>
        fareStatsTemplate.init = function(){
            minMaxStats.invoke()
        }
    </script>
    <template is="dom-repeat" items='{{stats.data}}' as='stat'>
        <paper-card class='driverStats' elevation="1">
            <div class="card-content">
                <div class="vertical-section">
                    <h4>Driver: <span>{{stat.0}}</span></h4><br/>
                    <div class="center horizontal layout">
                        <div class="stat-label">Avg</div>
                        <paper-slider pin value="{{stat.2}}" min='{{minMax.0}}' max='{{minMax.1}}'></paper-slider>
                        <div class="stat-value">$<span>{{stat.2}}</span></div>
                    </div>
                    
                    <div class="center horizontal layout">
                        <div class="stat-label">Min</div>
                        <paper-slider pin value="{{stat.3}}" min='{{minMax.4}}' max='{{minMax.5}}' class="orange"></paper-slider>
                        <div class="stat-value">$<span>{{stat.3}}</span></div>
                    </div>
                    
                    <div class="center horizontal layout">
                        <div class="stat-label">Max</div>
                        <paper-slider pin value="{{stat.4}}" min='{{minMax.2}}' max='{{minMax.3}}' class="red"></paper-slider>
                        <div class="stat-value">$<span>{{stat.4}}</span></div>
                    </div>
                </div>
            </div>
        </paper-card>
    </template>
</template>